In [1]:
import requests

### handing the url of the websit we want to scrap

In [2]:
standing_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [10]:
data = requests.get(standing_url)

In [11]:
data

<Response [429]>

### using a library for parsing url 

In [12]:
from bs4 import BeautifulSoup

In [6]:
soup = BeautifulSoup(data.text, 'lxml')

In [7]:
#using scc selector  to find the table of matches

table = soup.select('table.stats_table')[0]

IndexError: list index out of range

In [ ]:
links = table.find_all('a')

In [ ]:
links = [l.get("href") for l in links]

In [ ]:
links =[l for l in links if '/squads/' in l]

In [ ]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [ ]:
team_url = team_urls[0]


In [ ]:
data = requests.get(team_url)

In [ ]:
data

In [13]:
import pandas as pd

In [ ]:
matches = pd.read_html(data.text, match="Scores & Fixtures ")

In [ ]:
matches[0]

In [ ]:
soup = BeautifulSoup(data.text)

In [ ]:
links = soup.find_all('a')
links = [l.get("href") for l in links]
links

In [ ]:
links = [l for l in links if l and 'all_comps/shooting/' in l ]

In [ ]:
links

In [ ]:
data = requests.get(f"https://fbref.com{links[0]}")


In [ ]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [ ]:
shooting.head()

In [ ]:
shooting.columns = shooting.columns.droplevel()

In [ ]:
shooting.head()

In [ ]:
shooting.columns

In [ ]:
matches[0].head()

In [ ]:
shooting["Date"]

In [ ]:
team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on= "Date")

In [ ]:
team_data

In [ ]:
shooting.shape

In [14]:
years = list(range(2023, 2020, -1))

In [15]:
years

[2023, 2022, 2021]

In [16]:
all_matches = []
all_upcoming_matches = []

In [17]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [ ]:
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup

standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

all_matches = []
all_upcoming_matches = []

years = list(range(2023, 2020, -1))

for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    print(data)
    
    sleeptime = 10
    while data.status_code == 429:
        time.sleep(sleeptime)
        data = requests.get(standings_url)
        print(data)
        sleeptime = sleeptime*2
        
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links =[l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url= f"https://fbref.com{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats","").replace("-"," ")

        data = requests.get(team_url)
        print(data)
        
        sleeptime = 10
        while data.status_code == 429: 
            time.sleep(sleeptime)
            data = requests.get(team_url)
            sleeptime = sleeptime*2
            print(data)
                
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

        upcomming_matches = matches[(matches['Date'].notnull()) & (matches['Result'].isnull()) & (matches['Comp'] == "Premier League")]
        
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l ]
        print(links)
        data = requests.get(f"https://fbref.com{links[0]}")
        print(data)
        
        sleeptime = 10
        while data.status_code == 429:
            time.sleep(sleeptime)
            data = requests.get(team_url)
            print(data)
            sleeptime = sleeptime*2
                    
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()

        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on= "Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        
        upcomming_matches["Season"] = year
        upcomming_matches["Team"] = team_name
        all_upcoming_matches.append(upcomming_matches)

        #to avoid getting kicked from the websiteb (some website don't allow quick scrapping saying it slows down their websites)
        time.sleep(1)

match_df = pd.concat(all_matches)
upcoming_matches_df = pd.concat(all_upcoming_matches)

match_df.columns = [c.lower() for c in match_df.columns]
upcoming_matches_df.columns = [c.lower() for c in match_df.columns]

match_df.to_csv("matches.csv")
upcoming_matches_df.to_csv("upcomig_matches.csv")

<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>


In [ ]:
match_df = pd.concat(all_matches)
upcoming_matches_df = pd.concat(all_upcoming_matches)

In [ ]:
match_df.columns = [c.lower() for c in match_df.columns]
upcoming_matches_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df

In [ ]:
match_df.to_csv("matches.csv")

In [ ]:
upcoming_matches_df